In [1]:
"""Example of Estimator for DNN-based text classification with DBpedia data."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import numpy as np
import pandas
from sklearn import metrics
import tensorflow as tf

import csv

import os

import tarfile



import numpy as np
from tensorflow.contrib.learn.python.learn.datasets import text_datasets


from tensorflow.contrib.learn.python.learn.datasets import base

from tensorflow.python.platform import gfile

In [2]:
FLAGS = None
MAX_DOCUMENT_LENGTH = 10
EMBEDDING_SIZE = 50
n_words = 0
MAX_LABEL = 15
WORDS_FEATURE = 'words'  # Name of the input words feature.

In [3]:
def estimator_spec_for_softmax_classification(logits, labels, mode):

  """Returns EstimatorSpec instance for softmax classification."""
  predicted_classes = tf.argmax(logits, 1)
    
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions={
            'class': predicted_classes,
            'prob': tf.nn.softmax(logits)
        })

  onehot_labels = tf.one_hot(labels, MAX_LABEL, 1, 0)

  loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

  eval_metric_ops = {
      'accuracy': tf.metrics.accuracy(
          labels=labels, predictions=predicted_classes)
  }

  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
def bag_of_words_model(features, target):

  """A bag-of-words model. Note it disregards the word order in the text."""
  bow_column = tf.feature_column.categorical_column_with_identity(WORDS_FEATURE, num_buckets=n_words)

  bow_embedding_column = tf.feature_column.embedding_column(bow_column, dimension=EMBEDDING_SIZE)

  bow = tf.feature_column.input_layer(features, feature_columns=[bow_embedding_column])

  logits = tf.layers.dense(bow, MAX_LABEL, activation=None)

  return estimator_spec_for_softmax_classification(
      logits=logits, labels=labels, mode=mode)


In [5]:
def rnn_model(features, labels, mode):

  """RNN model to predict from sequence of words to a class."""
  # Convert indexes of words into embeddings.
  # This creates embeddings matrix of [n_words, EMBEDDING_SIZE] and then
  # maps word indexes of the sequence into [batch_size, sequence_length,
  # EMBEDDING_SIZE].

  word_vectors = tf.contrib.layers.embed_sequence(
      features[WORDS_FEATURE], vocab_size=n_words, embed_dim=EMBEDDING_SIZE)

  # Split into list of embedding per word, while removing doc length dim.
  # word_list results to be a list of tensors [batch_size, EMBEDDING_SIZE].

  word_list = tf.unstack(word_vectors, axis=1)

  # Create a Gated Recurrent Unit cell with hidden size of EMBEDDING_SIZE.
  cell = tf.contrib.rnn.GRUCell(EMBEDDING_SIZE)

  # Create an unrolled Recurrent Neural Networks to length of
  # MAX_DOCUMENT_LENGTH and passes word_list as inputs for each unit.
  _, encoding = tf.contrib.rnn.static_rnn(cell, word_list, dtype=tf.float32)

  # Given encoding of RNN, take encoding of last step (e.g hidden size of the
  # neural network of last step) and pass it as features for softmax
  # classification over output classes.

  logits = tf.layers.dense(encoding, MAX_LABEL, activation=None)

  return estimator_spec_for_softmax_classification(
      logits=logits, labels=labels, mode=mode)

In [6]:
#def shrink_csv(filename, ratio):

#  """Create a smaller dataset of only 1/ratio of original data."""

 # filename_small = filename.replace('.', '_medium.')

  #with gfile.Open(filename_small, 'w') as csv_file_small:

   # writer = csv.writer(csv_file_small)

    #with gfile.Open(filename) as csv_file:

     # reader = csv.reader(csv_file)

      #i = 0

     # for row in reader:

      #  if i % ratio == 0:

       #   writer.writerow(row)

       # i += 1

In [7]:
#data_dir = os.path.join(os.getenv('TF_EXP_BASE_DIR', ''), 'dbpedia_data')


#train_path = os.path.join(data_dir, 'dbpedia_csv/train.csv')

#test_path = os.path.join(data_dir, 'dbpedia_csv/test.csv')

#train_path = os.path.join(data_dir, 'dbpedia_csv', 'train.csv')
#test_path = os.path.join(data_dir, 'dbpedia_csv', 'test.csv')

# Reduce the size of original data by a factor of 1000.

#shrink_csv(train_path, 8)

#shrink_csv(test_path, 8)



#train_path = train_path.replace('train.csv', 'train_medium.csv')

#test_path = test_path.replace('test.csv', 'test_medium.csv')

In [8]:
def load_dbpedia(size='small', test_with_fake_data=False):
  """Get DBpedia datasets from CSV files."""
  if not test_with_fake_data:
    data_dir = os.path.join(os.getenv('TF_EXP_BASE_DIR', ''), 'dbpedia_data')
    text_datasets.maybe_download_dbpedia(data_dir)

    train_path = os.path.join(data_dir, 'dbpedia_csv', 'train.csv')
    test_path = os.path.join(data_dir, 'dbpedia_csv', 'test.csv')

    if size == 'small':
      # Reduce the size of original data by a factor of 1000.
      base.shrink_csv(train_path, 7)
      base.shrink_csv(test_path, 7)
      train_path = train_path.replace('train.csv', 'train_small.csv')
      test_path = test_path.replace('test.csv', 'test_small.csv')

  else:
    module_path = os.path.dirname(__file__)
    train_path = os.path.join(module_path, 'data', 'text_train.csv')
    test_path = os.path.join(module_path, 'data', 'text_test.csv')

  train = base.load_csv_without_header(
      train_path, target_dtype=np.int32, features_dtype=np.str, target_column=0)
  test = base.load_csv_without_header(
      test_path, target_dtype=np.int32, features_dtype=np.str, target_column=0)

  return base.Datasets(train=train, validation=None, test=test)

In [9]:
def main(unused_argv):

  global n_words

  # Prepare training and testing data
#  dbpedia = tf.contrib.learn.datasets.load_dataset(
#      'dbpedia',test_with_fake_data=FLAGS.test_with_fake_data)

  dbpedia = load_dbpedia(size='small', test_with_fake_data=FLAGS.test_with_fake_data)

  x_train = pandas.Series(dbpedia.train.data[:,1])
  y_train = pandas.Series(dbpedia.train.target)
  x_test = pandas.Series(dbpedia.test.data[:,1])
  y_test = pandas.Series(dbpedia.test.target)

  # Process vocabulary
  vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(
      MAX_DOCUMENT_LENGTH)

  x_train = np.array(list(vocab_processor.fit_transform(x_train)))
  x_test = np.array(list(vocab_processor.transform(x_test)))

  n_words = len(vocab_processor.vocabulary_)
  print('Total words: %d' % n_words)

  # Build model
  # Switch between rnn_model and bag_of_words_model to test different models.
  model_fn = rnn_model

  if FLAGS.bow_model:
    # Subtract 1 because VocabularyProcessor outputs a word-id matrix where word
    # ids start from 1 and 0 means 'no word'. But
    # categorical_column_with_identity assumes 0-based count and uses -1 for
    # missing word.
    x_train -= 1
    x_test -= 1
    model_fn = bag_of_words_model

  classifier = tf.estimator.Estimator(model_fn=model_fn)

  # Train.
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={WORDS_FEATURE: x_train},
      y=y_train,
      batch_size=len(x_train),
      num_epochs=None,
      shuffle=True)

  classifier.train(input_fn=train_input_fn, steps=100)

  # Predict.
  test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={WORDS_FEATURE: x_test},
      y=y_test,
      num_epochs=1,
      shuffle=False)

  predictions = classifier.predict(input_fn=test_input_fn)
  y_predicted = np.array(list(p['class'] for p in predictions))
  y_predicted = y_predicted.reshape(np.array(y_test).shape)

  # Score with sklearn.
  score = metrics.accuracy_score(y_test, y_predicted)
  print('Accuracy (sklearn): {0:f}'.format(score))

  # Score with tensorflow.
  scores = classifier.evaluate(input_fn=test_input_fn)
  print('Accuracy (tensorflow): {0:f}'.format(scores['accuracy']))

In [10]:
if __name__ == '__main__':

  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--test_with_fake_data',
      default=False,
      help='Test the example code with fake data.',
      action='store_true')

  parser.add_argument(
      '--bow_model',
      default=False,
      help='Run with BOW model instead of RNN.',
      action='store_true')

  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Total words: 228987
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Vilma\AppData\Local\Temp\tmp6k5mi3u1\model.ckpt.
INFO:tensorflow:loss = 2.70789, step = 1
INFO:tensorflow:Saving checkpoints for 100 into C:\Users\Vilma\AppData\Local\Temp\tmp6k5mi3u1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.00101484.
INFO:tensorflow:Restoring parameters from C:\Users\Vilma\AppData\Local\Temp\tmp6k5mi3u1\model.ckpt-100
Accuracy (sklearn): 0.800400
INFO:tensorflow:Starting evaluation at 2017-10-08-23:21:34
INFO:tensorflow:Restoring parameters from C:\Users\Vilma\AppData\Local\Temp\tmp6k5mi3u1\model.ckpt-100
INFO:tensorflow:Finished evaluation at 2017-10-08-23:21:35
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.8004, global_step = 100, loss = 1.13198
Accuracy (tensorflow): 0.800400


SystemExit: 

C:\Users\Vilma\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2855: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
